In [1]:
device_id = 1
device = 'cuda:' + str(device_id)
#device = 'cpu'

import time
import os
import torch

log_dir = os.path.join('./log/question_classifier', time.strftime('%Y%m%d_%H%M%S', time.localtime(time.time())))
print(log_dir)

./log/question_classifier/20210617_235128


In [2]:
from pkg.parse import *
from pkg.words import *
from problems import *

problems = [P1_1_1, P1_1_2, P1_1_3, P1_1_4, P1_1_5, P1_1_6, P1_1_7, P1_1_8, P1_1_9, P1_1_10, P1_1_11, P1_1_12, 
            P1_2_1, P1_2_2, P1_3_1, P1_4_1, 
            P2_1_1, P2_2_2, P2_3_1, 
            P3_1_1, P3_2_1, P3_2_2, P3_3_1, 
            P4_1_1, P4_2_1, P4_2_2, P4_3_1, 
            P5_1_1, P5_2_1, P5_3_1,
            P6_1_1, P6_3_1, P6_4_1,
            P7_1_1, P7_1_2, P7_3_1,
            P8_1_1, P8_2_1, P8_3_1, 
            P9_1_1, P9_2_1, P9_2_2, P9_3_1, P9_3_2]

In [3]:
from pkg.trainer_question_classifier import Hyper as hp

# dataset
hp.add_bos = False
hp.add_eos = False

hp.batch_size = None
hp.batch_type = 'normal'
hp.ds_batch_size = 256

# train
hp.num_workers = 2

hp.steps_log = 10
hp.steps_eval = 50
hp.steps_save = 10000

hp.weight_decay = 0.000001
hp.initial_lr = 0.0001
hp.final_lr = 0.00001
hp.lr_decay_factor = 0.99
hp.lr_patience = 300
hp.ema = 0.99
hp.grad_norm_max = 10.0

hp.adam_alpha = 2e-4
hp.adam_betas = (0.5, 0.9)
hp.adam_eps = 1e-6

hp.vocab_size = 512
hp.add_space_token = False

hp.num_problems = len(problems)
hp.problems = problems


Instructions for updating:
non-resource variables are not supported in the long term


In [4]:
from pkg.dataset import ProblemDataset, QuestionDataset, read_questions, write_questions

dir_question = 'data/question'
_prefix = 'question_'
_ids = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
path_questions = [os.path.join(dir_question, _prefix + str(i) + '.txt') for i in _ids]
metas = [read_questions(path) for path in path_questions]
meta = []
for m in metas:
    meta += m
len_train = int(len(meta) * 0.8)
meta_train = meta[:len_train]
meta_val = meta[len_train:]
print(len(meta), len(meta_train), len(meta_val))

2816000 2252800 563200


In [5]:
from pkg.vocab import Vocab, CharVocab, SPVocab
dir_token = 'tokenization'
filename = 'prob'
filename += '_' + str(hp.vocab_size)
if hp.add_space_token:
    filename += '_'
filename += '.model'
path_model = os.path.join(dir_token, filename)
print(path_model)
vocab = SPVocab(path_model)
print(vocab.vocab_size())

tokenization/prob_512.model
512


In [6]:
start = time.time()
dsTrain = QuestionDataset(meta_train, vocab, batch_size=hp.ds_batch_size)
dsVal = QuestionDataset(meta_val, vocab, batch_size=hp.ds_batch_size)
print(time.time() - start)

653.4771099090576


In [7]:
from pkg.models.model import QuestionClassifier
from pkg.models.config import Config
from pkg.models.extractor import AverageExtractor, RNNExtractor
from pkg.models.encoders_conv import ConvEncoder, HighwayEncoder
from pkg.models.embedding import Embed, Regressor

cfg = QuestionClassifier.default_config()
cfg.text_embed.num_symbols = hp.vocab_size
cfg.regressor.num_symbols = hp.num_problems
cfg.extractor = RNNExtractor.default_config()
model = cfg.create_object().to(device)

In [9]:
from pkg.trainer_question_classifier import Trainer_QuestionClassifier

trainer = Trainer_QuestionClassifier(model, dsTrain, dsVal, hp=hp, log_dir=log_dir)

---------------------- hp -------------------

__module__ pkg.trainer
steps_log 10
steps_eval 50
steps_save 10000
save_model_only True
max_lr 0.001
base_lr 0.001
lr_decay_factor 0.99
lr_patience 300
scheduler_mode triangular2
step_size_up 10
ema 0.99
max_grad_norm 10.0
adam_alpha 0.0002
adam_betas (0.5, 0.9)
adam_eps 1e-06
weight_decay 1e-06
__dict__ <attribute '__dict__' of 'Hyper' objects>
__weakref__ <attribute '__weakref__' of 'Hyper' objects>
__doc__ None
add_bos False
add_eos False
batch_size None
batch_type normal
ds_batch_size 256
num_workers 4
initial_lr 0.0001
final_lr 1e-05
grad_norm_max 10.0
vocab_size 512
add_space_token False
num_problems 44
problems [<class 'problems.P1_1_1'>, <class 'problems.P1_1_2'>, <class 'problems.P1_1_3'>, <class 'problems.P1_1_4'>, <class 'problems.P1_1_5'>, <class 'problems.P1_1_6'>, <class 'problems.P1_1_7'>, <class 'problems.P1_1_8'>, <class 'problems.P1_1_9'>, <class 'problems.P1_1_10'>, <class 'problems.P1_1_11'>, <class 'problems.P1_1_12'>,

In [ ]:
trainer.train()

step: 10 train: 0.08227705955505371 elapsed, loss: 3.6839209
step: 20 train: 0.08821368217468262 elapsed, loss: 3.4062886
step: 30 train: 0.07718873023986816 elapsed, loss: 2.997042
step: 40 train: 0.07712173461914062 elapsed, loss: 2.467427
step: 50 train: 0.07581782341003418 elapsed, loss: 2.2078242
step: 60 train: 0.07978153228759766 elapsed, loss: 1.7570738
step: 70 train: 0.07337403297424316 elapsed, loss: 1.6777754
step: 80 train: 0.08415603637695312 elapsed, loss: 1.2404146
step: 90 train: 0.08092045783996582 elapsed, loss: 1.2091846
step: 100 train: 0.08152079582214355 elapsed, loss: 1.0216315
step: 110 train: 0.08567500114440918 elapsed, loss: 0.89112204
step: 120 train: 0.0851280689239502 elapsed, loss: 0.8294809
step: 130 train: 0.08115625381469727 elapsed, loss: 0.7105387
step: 140 train: 0.07694387435913086 elapsed, loss: 0.6105763
step: 150 train: 0.07736968994140625 elapsed, loss: 0.5314468
step: 160 train: 0.08264875411987305 elapsed, loss: 0.42523825
step: 170 train: 0

step: 1280 train: 0.08230900764465332 elapsed, loss: 0.011225957
step: 1290 train: 0.07554841041564941 elapsed, loss: 1.5694965e-05
step: 1300 train: 0.07854652404785156 elapsed, loss: 1.8316168e-05
step: 1310 train: 0.08622312545776367 elapsed, loss: 2.4123434e-05
step: 1320 train: 0.08711004257202148 elapsed, loss: 1.5294994e-05
step: 1330 train: 0.08301091194152832 elapsed, loss: 5.2346622e-05
step: 1340 train: 0.08238673210144043 elapsed, loss: 1.0064088e-05
step: 1350 train: 0.07632875442504883 elapsed, loss: 7.904957e-06
step: 1360 train: 0.08085417747497559 elapsed, loss: 8.316585e-06
step: 1370 train: 0.08417344093322754 elapsed, loss: 7.806716e-06
step: 1380 train: 0.077728271484375 elapsed, loss: 0.0013406198
step: 1390 train: 0.08191394805908203 elapsed, loss: 1.9180377e-05
step: 1400 train: 0.08301186561584473 elapsed, loss: 0.0004538429
step: 1410 train: 0.07830953598022461 elapsed, loss: 0.0019284594
step: 1420 train: 0.0765688419342041 elapsed, loss: 1.1348265e-05
step: 

step: 2520 train: 0.08189201354980469 elapsed, loss: 2.6761504e-06
step: 2530 train: 0.0789797306060791 elapsed, loss: 3.5678845e-06
step: 2540 train: 0.07893228530883789 elapsed, loss: 2.5802224e-06
step: 2550 train: 0.07836484909057617 elapsed, loss: 2.9536805e-06
step: 2560 train: 0.09025120735168457 elapsed, loss: 2.7040774e-06
step: 2570 train: 0.07968711853027344 elapsed, loss: 3.418875e-06
step: 2580 train: 0.07780647277832031 elapsed, loss: 8.3577515e-06
step: 2590 train: 0.07709217071533203 elapsed, loss: 3.3462316e-06
step: 2600 train: 0.08466768264770508 elapsed, loss: 2.6058356e-06
step: 2610 train: 0.08156895637512207 elapsed, loss: 2.6458813e-06
step: 2620 train: 0.07698488235473633 elapsed, loss: 3.9008205e-06
step: 2630 train: 0.08096957206726074 elapsed, loss: 3.6232984e-06
step: 2640 train: 0.07982635498046875 elapsed, loss: 3.1450643e-06
step: 2650 train: 0.07782697677612305 elapsed, loss: 3.0621788e-06
step: 2660 train: 0.08789181709289551 elapsed, loss: 2.8326112e-

step: 3760 train: 0.08091545104980469 elapsed, loss: 3.6088609e-06
step: 3770 train: 0.07899808883666992 elapsed, loss: 4.730079e-06
step: 3780 train: 0.07857394218444824 elapsed, loss: 2.4670671e-06
step: 3790 train: 0.08114933967590332 elapsed, loss: 3.6833494e-06
step: 3800 train: 0.08415699005126953 elapsed, loss: 2.7981496e-06
step: 3810 train: 0.08741354942321777 elapsed, loss: 2.4773126e-06
step: 3820 train: 0.07451033592224121 elapsed, loss: 3.7350592e-06
step: 3830 train: 0.07959151268005371 elapsed, loss: 2.7664878e-06
step: 3840 train: 0.08043527603149414 elapsed, loss: 3.1646105e-06
step: 3850 train: 0.08030414581298828 elapsed, loss: 3.2521707e-06
step: 3860 train: 0.07507491111755371 elapsed, loss: 1.3646011e-05
step: 3870 train: 0.0764167308807373 elapsed, loss: 3.7182908e-06
step: 3880 train: 0.08387541770935059 elapsed, loss: 2.9825533e-06
step: 3890 train: 0.07720565795898438 elapsed, loss: 3.3788283e-06
step: 3900 train: 0.0798637866973877 elapsed, loss: 2.8335414e-0

step: 5000 train: 0.07777833938598633 elapsed, loss: 3.4845343e-06
step: 5010 train: 0.07867622375488281 elapsed, loss: 3.1022278e-06
step: 5020 train: 0.08187723159790039 elapsed, loss: 2.5196878e-06
step: 5030 train: 0.08135986328125 elapsed, loss: 3.2349344e-06
step: 5040 train: 0.08207440376281738 elapsed, loss: 2.5508866e-06
step: 5050 train: 0.07750535011291504 elapsed, loss: 2.7250444e-06
step: 5060 train: 0.07976889610290527 elapsed, loss: 2.8326112e-06
step: 5070 train: 0.07912993431091309 elapsed, loss: 3.5087478e-06
step: 5080 train: 0.07447052001953125 elapsed, loss: 3.8943103e-06
step: 5090 train: 0.08267092704772949 elapsed, loss: 3.1534514e-06
step: 5100 train: 0.07971453666687012 elapsed, loss: 2.4107223e-06
step: 5110 train: 0.0777275562286377 elapsed, loss: 3.887785e-06
step: 5120 train: 0.07452988624572754 elapsed, loss: 4.933622e-06
step: 5130 train: 0.08531045913696289 elapsed, loss: 2.8745171e-06
step: 5140 train: 0.08206486701965332 elapsed, loss: 2.8400623e-06
s

...

step: 160200 train: 0.0794379711151123 elapsed, loss: 1.912003e-06
step: 160210 train: 0.08333873748779297 elapsed, loss: 0.00019899524
step: 160220 train: 0.08751201629638672 elapsed, loss: 0.00015009799
step: 160230 train: 0.07876944541931152 elapsed, loss: 0.0013002092
step: 160240 train: 0.08741164207458496 elapsed, loss: 0.00017158387
step: 160250 train: 0.08288979530334473 elapsed, loss: 3.7902857e-05
step: 160260 train: 0.09296488761901855 elapsed, loss: 5.1548093e-05
step: 160270 train: 0.0779106616973877 elapsed, loss: 1.8266895e-05
step: 160280 train: 0.08136606216430664 elapsed, loss: 1.3658563e-05
step: 160290 train: 0.07566499710083008 elapsed, loss: 1.1960332e-05
step: 160300 train: 0.08191108703613281 elapsed, loss: 9.961803e-06
step: 160310 train: 0.08076810836791992 elapsed, loss: 4.0717287e-06
step: 160320 train: 0.08401679992675781 elapsed, loss: 3.781621e-06
step: 160330 train: 0.08750224113464355 elapsed, loss: 3.0007127e-06
step: 160340 train: 0.07939338684082031 

step: 161400 train: 0.07727551460266113 elapsed, loss: 1.9976842e-06
step: 161410 train: 0.08350968360900879 elapsed, loss: 1.9404083e-06
step: 161420 train: 0.08185434341430664 elapsed, loss: 1.570674e-06
step: 161430 train: 0.08219194412231445 elapsed, loss: 1.7797554e-06
step: 161440 train: 0.08874034881591797 elapsed, loss: 1.3639207e-06
step: 161450 train: 0.08370280265808105 elapsed, loss: 1.8006814e-06
step: 161460 train: 0.0865941047668457 elapsed, loss: 1.4477331e-06
step: 161470 train: 0.08194494247436523 elapsed, loss: 1.3266679e-06
step: 161480 train: 0.07960081100463867 elapsed, loss: 1.5767275e-06
step: 161490 train: 0.0875694751739502 elapsed, loss: 1.654017e-06
step: 161500 train: 0.0904233455657959 elapsed, loss: 1.573002e-06
step: 161510 train: 0.08127307891845703 elapsed, loss: 1.9003605e-06
step: 161520 train: 0.08335089683532715 elapsed, loss: 1.2260851e-06
step: 161530 train: 0.08161783218383789 elapsed, loss: 1.6386605e-06
step: 161540 train: 0.07705211639404297 

step: 162600 train: 0.08514189720153809 elapsed, loss: 1.3103697e-06
step: 162610 train: 0.08316254615783691 elapsed, loss: 1.3904635e-06
step: 162620 train: 0.0834801197052002 elapsed, loss: 1.3411034e-06
step: 162630 train: 0.08102917671203613 elapsed, loss: 1.3606609e-06
step: 162640 train: 0.07862567901611328 elapsed, loss: 1.8486733e-06
step: 162650 train: 0.08090972900390625 elapsed, loss: 1.3988453e-06
step: 162660 train: 0.07681155204772949 elapsed, loss: 1.85007e-06
step: 162670 train: 0.07990455627441406 elapsed, loss: 1.2484364e-06
step: 162680 train: 0.07854819297790527 elapsed, loss: 1.8030383e-06
step: 162690 train: 0.08087038993835449 elapsed, loss: 1.4724196e-06
step: 162700 train: 0.07855677604675293 elapsed, loss: 1.4947714e-06
step: 162710 train: 0.0842275619506836 elapsed, loss: 1.4607779e-06
step: 162720 train: 0.08298134803771973 elapsed, loss: 1.6414542e-06
step: 162730 train: 0.08691573143005371 elapsed, loss: 1.3750966e-06
step: 162740 train: 0.0795407295227050

step: 163800 train: 0.0883951187133789 elapsed, loss: 2.474957e-06
step: 163810 train: 0.08208155632019043 elapsed, loss: 2.114565e-06
step: 163820 train: 0.07964181900024414 elapsed, loss: 1.9790582e-06
step: 163830 train: 0.08155941963195801 elapsed, loss: 2.0191044e-06
step: 163840 train: 0.07841157913208008 elapsed, loss: 2.0870912e-06
step: 163850 train: 0.08080315589904785 elapsed, loss: 2.005135e-06
step: 163860 train: 0.07897496223449707 elapsed, loss: 2.057289e-06
step: 163870 train: 0.07949090003967285 elapsed, loss: 1.5781246e-06
step: 163880 train: 0.07756280899047852 elapsed, loss: 2.2794084e-06
step: 163890 train: 0.07883715629577637 elapsed, loss: 1.9818522e-06
step: 163900 train: 0.08769559860229492 elapsed, loss: 1.4081586e-06
step: 163910 train: 0.08916401863098145 elapsed, loss: 2.3287687e-06
step: 163920 train: 0.09201741218566895 elapsed, loss: 1.9394768e-06
step: 163930 train: 0.08272600173950195 elapsed, loss: 1.6712568e-06
step: 163940 train: 0.07655572891235352

step: 165000 train: 0.07748174667358398 elapsed, loss: 1.4482051e-06
step: 165010 train: 0.07908987998962402 elapsed, loss: 1.976264e-06
step: 165020 train: 0.07766580581665039 elapsed, loss: 1.3639208e-06
step: 165030 train: 0.08385443687438965 elapsed, loss: 1.2647349e-06
step: 165040 train: 0.08066153526306152 elapsed, loss: 1.6265521e-06
step: 165050 train: 0.08824729919433594 elapsed, loss: 9.913922e-07
step: 165060 train: 0.07853174209594727 elapsed, loss: 1.8523986e-06
step: 165070 train: 0.08154749870300293 elapsed, loss: 1.4044322e-06
step: 165080 train: 0.0774838924407959 elapsed, loss: 1.5697426e-06
step: 165090 train: 0.09171342849731445 elapsed, loss: 1.1119982e-06
step: 165100 train: 0.08112788200378418 elapsed, loss: 1.4803359e-06
step: 165110 train: 0.07894039154052734 elapsed, loss: 1.7080439e-06
step: 165120 train: 0.08751058578491211 elapsed, loss: 1.5227109e-06
step: 165130 train: 0.08312630653381348 elapsed, loss: 1.4258535e-06
step: 165140 train: 0.090494155883789